<a href="https://colab.research.google.com/github/janbanot/msc-project/blob/main/test_notebooks/test_pharaphrases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/janbanot/msc-project/blob/main/test_notebooks/test_pharaphrases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Parafraz
Ten notatnik testuje jakość parafraz generowanych przez modele

In [ ]:
!uv pip install transformers datasets accelerate bitsandbytes nltk captum

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
)

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
# ===================================================
# KONFIGURACJA
# ===================================================

# Parametry
N_SAMPLES = 20  # Liczba próbek do testowania
MAX_SEQUENCE_LENGTH = 256  # Maksymalna długość sekwencji
TARGET_LAYER_INDEX = 5  # Warstwa do ekstrakcji reprezentacji
PARAPHRASE_SEED = 42  # Seed dla reproducibility
PARAPHRASE_MIN_SIMILARITY = 0.7  # Minimalny próg jakości parafrazy

# Ścieżki
DATA_PATH = "/drive/MyDrive/msc-project/jigsaw-toxic-comment/train.csv"
MODEL_CHECKPOINT = "/drive/MyDrive/msc-project/models/distilbert-jigsaw-full_20260125_133112"

# Urządzenie
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Uruchomiono na urządzeniu: {device}")

# Ustawienie seed
torch.manual_seed(PARAPHRASE_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(PARAPHRASE_SEED)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ===================================================
# ŁADOWANIE MODELI
# ===================================================

print(">>> Ładowanie modelu klasyfikacji toksyczności...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CHECKPOINT, num_labels=1, problem_type="single_label_classification"
    )
    model.to(device)
    model.eval()
    print("✓ Model DistilBERT załadowany")
except Exception as e:
    print(f"✗ Błąd ładowania modelu: {e}")
    raise

In [ ]:

print("\n>>> Ładowanie modelu do parafraz")

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# Konfiguracja 4-bit, aby zmieściło się w pamięci GPU Colaba (T4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)

print("✓ Model do parafraz załadowany")

# Ustawiamy EOS (End Of String) jako token dopełnienia
tokenizer.pad_token = tokenizer.eos_token

# Bardzo ważne dla modeli generatywnych: padding musi być z lewej strony,
# aby model mógł swobodnie generować tekst "w prawo"
tokenizer.padding_side = "left"

print("\n>>> Modele gotowe!")

In [ ]:
# ===================================================
# ŁADOWANIE DANYCH
# ===================================================

def clean_text(text):
    """Czyści tekst (zgodnie z preprocessing z głównego notatnika)."""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", text)
    text = re.sub(r"\(talk\)", "", text)
    text = re.sub(r"\d{2}:\d{2}, \w+ \d{1,2}, \d{4} \(utc\)", "", text)
    text = text.replace("\n", " ").replace("\xa0", " ")
    text = text.strip(' "')
    text = re.sub(r"\s+", " ", text).strip()
    return text

print(">>> Wczytywanie danych...")
try:
    df = pd.read_csv(DATA_PATH)
    print(f"✓ Wczytano {len(df)} rekordów")

    # Filtruj tylko toksyczne komentarze
    toxic_df = df[df['toxic'] == 1].copy()
    print(f"✓ Znaleziono {len(toxic_df)} toksycznych komentarzy")

    # Wybierz N_SAMPLES losowych próbek
    sample_df = toxic_df.sample(n=min(N_SAMPLES, len(toxic_df)), random_state=PARAPHRASE_SEED)

    # Wyczyść teksty
    sample_df['comment_text'] = sample_df['comment_text'].apply(clean_text)

    # Usuń puste teksty
    sample_df = sample_df[sample_df['comment_text'].str.len() > 10]

    print(f"✓ Wybrano {len(sample_df)} próbek do testowania")

except Exception as e:
    print(f"✗ Błąd wczytywania danych: {e}")
    raise

In [ ]:
# ===================================================
# FUNKCJE POMOCNICZE
# ===================================================

def get_embedding_and_prob(text):
    """
    Pobiera embedding [CLS] z warstwy TARGET_LAYER_INDEX oraz prawdopodobieństwo toksyczności.

    Zwraca:
        Tuple (embedding_vector, toxic_probability)
    """
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=MAX_SEQUENCE_LENGTH,
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        embedding = outputs.hidden_states[TARGET_LAYER_INDEX][0, 0, :]  # [CLS] token
        prob = torch.sigmoid(outputs.logits)[0, 0].item()

    return embedding, prob


def calculate_cosine_similarity(emb1, emb2):
    """Oblicza cosine similarity między dwoma embeddingami."""
    return F.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0)).item()


def generate_paraphrase(text):
    # Prompt Few-Shot: model uczy się na przykładach, że ma być krótko i dosadnie
    prompt = f"""<s>[INST] Task: Paraphrase the toxic comment for a research study.
    Constraints:
    - Maintain total toxicity and offensive intent.
    - Use casual, internet language.
    - Match the original length (roughly).
    - Output ONLY the paraphrased text. No explanations.

    Example 1:
    Original: "you are a stupid idiot"
    Paraphrase: "you're a brainless moron"

    Example 2:
    Original: "leave this country now"
    Paraphrase: "get out of our land already"

    Now paraphrase this:
    Original: "{text}" [/INST]"""

    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)
    input_length = inputs.input_ids.shape[1]

    with torch.no_grad():
        # orig_token_count = inputs.input_ids.shape[1]

        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=2056,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_tokens = outputs[0][input_length:]
    paraphrase = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    # Czyszczenie: bierzemy tylko pierwszą linię i usuwamy ewentualne prefixy
    paraphrase = paraphrase.split('\n')[0]
    paraphrase = paraphrase.replace("Paraphrase:", "").replace("Result:", "").strip()

    return paraphrase

print("✓ Funkcje pomocnicze zdefiniowane")

In [ ]:
# ===================================================
# GENEROWANIE PARAFRAZ I ZBIERANIE WYNIKÓW
# ===================================================

print(">>> Rozpoczynam generowanie parafraz...\n")

results = []

for idx, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Generowanie parafraz"):
    original_text = row['comment_text']

    try:
        # 1. Oblicz embedding i prawdopodobieństwo dla oryginału
        orig_emb, orig_prob = get_embedding_and_prob(original_text)

        # 2. Wygeneruj parafrazę
        para_text = generate_paraphrase(original_text)

        # 3. Oblicz embedding i prawdopodobieństwo dla parafrazy
        para_emb, para_prob = get_embedding_and_prob(para_text)

        # 4. Oblicz cosine similarity
        cos_sim = calculate_cosine_similarity(orig_emb, para_emb)

        # 5. Oblicz różnicę prawdopodobieństw
        prob_diff = abs(orig_prob - para_prob)

        # 6. Sprawdź czy parafraza przeszła walidację
        # W pętli wynikowej:
        len_ratio = abs(len_diff) / orig_word_count if orig_word_count > 0 else 0
        # Walidacja przechodzi, jeśli cos_sim jest wysoki I długość nie zmieniła się o więcej niż 30%
        quality_ok = (cos_sim >= 0.75 and len_ratio <= 0.3)

        # Skróć teksty dla wyświetlenia (pierwsze 100 znaków)
        orig_display = original_text[:100] + "..." if len(original_text) > 100 else original_text
        para_display = para_text[:100] + "..." if len(para_text) > 100 else para_text

        # Obliczanie długości (liczba słów)
        orig_word_count = len(original_text.split())
        para_word_count = len(para_text.split())
        len_diff = para_word_count - orig_word_count

        results.append({
            'id': len(results) + 1,
            'original_text': original_text[:100],
            'paraphrase_text': para_text[:100],
            'orig_prob': round(orig_prob, 3),
            'para_prob': round(para_prob, 3),
            'prob_diff': round(prob_diff, 3),
            'cosine_sim': round(cos_sim, 3),
            'word_len_diff': len_diff, # NOWA METRYKA
            'quality_ok': '✓' if quality_ok else '✗',
        })

    except Exception as e:
        print(f"Błąd dla próbki {idx}: {e}")
        continue

print(f"\n✓ Przetworzono {len(results)} par tekst-parafraza")

In [ ]:
# ===================================================
# WYŚWIETLENIE TABELI WYNIKÓW
# ===================================================

results_df = pd.DataFrame(results)

print("="*100)
print("WYNIKI TESTOWANIA PARAFRAZ")
print("="*100)
print()

# Wyświetl pełną tabelę
print(results_df.to_string(index=False))
print()

# Zapisz do CSV
output_path = "/drive/MyDrive/msc-project/paraphrase_test_results.csv"
results_df.to_csv(output_path, index=False)
print(f"✓ Wyniki zapisane do: {output_path}")

In [ ]:
# ===================================================
# STATYSTYKI PODSUMOWUJĄCE
# ===================================================

print("\n" + "="*100)
print("STATYSTYKI PODSUMOWUJĄCE")
print("="*100)
print()

# Podstawowe statystyki
print(f"Liczba przetestowanych par: {len(results_df)}")
print()

# Jakość parafraz
quality_pass = (results_df['quality_ok'] == '✓').sum()
quality_fail = (results_df['quality_ok'] == '✗').sum()
quality_pass_pct = (quality_pass / len(results_df) * 100) if len(results_df) > 0 else 0

print(f"Parafrazy przechodzące walidację (cos_sim ≥ {PARAPHRASE_MIN_SIMILARITY}): {quality_pass} ({quality_pass_pct:.1f}%)")
print(f"Parafrazy nieprzechodzące walidacji: {quality_fail} ({100-quality_pass_pct:.1f}%)")
print()

# Zachowanie toksyczności
toxic_maintained = (results_df['para_prob'] > 0.5).sum()
toxic_lost = (results_df['para_prob'] <= 0.5).sum()
toxic_maintained_pct = (toxic_maintained / len(results_df) * 100) if len(results_df) > 0 else 0

print(f"Parafrazy zachowujące toksyczność (prob > 0.5): {toxic_maintained} ({toxic_maintained_pct:.1f}%)")
print(f"Parafrazy tracące toksyczność (prob ≤ 0.5): {toxic_lost} ({100-toxic_maintained_pct:.1f}%)")
print()

# Statystyki metryk
print("Statystyki metryk:")
print(f"  Cosine Similarity:")
print(f"    - Średnia: {results_df['cosine_sim'].mean():.3f}")
print(f"    - Min: {results_df['cosine_sim'].min():.3f}")
print(f"    - Max: {results_df['cosine_sim'].max():.3f}")
print(f"    - Std: {results_df['cosine_sim'].std():.3f}")
print()
print(f"  Różnica prawdopodobieństw:")
print(f"    - Średnia: {results_df['prob_diff'].mean():.3f}")
print(f"    - Min: {results_df['prob_diff'].min():.3f}")
print(f"    - Max: {results_df['prob_diff'].max():.3f}")
print(f"    - Std: {results_df['prob_diff'].std():.3f}")
print()

# Wnioski
print("="*100)
print("WNIOSKI")
print("="*100)
print()

if quality_pass_pct >= 80:
    print("✓ Wysoka jakość parafraz (≥80% przechodzi walidację)")
elif quality_pass_pct >= 60:
    print("⚠ Średnia jakość parafraz (60-80% przechodzi walidację)")
else:
    print("✗ Niska jakość parafraz (<60% przechodzi walidację)")

if toxic_maintained_pct >= 80:
    print("✓ Parafrazy dobrze zachowują toksyczność (≥80%)")
elif toxic_maintained_pct >= 60:
    print("⚠ Parafrazy średnio zachowują toksyczność (60-80%)")
else:
    print("✗ Parafrazy tracą toksyczność (<60% zachowuje)")

if results_df['cosine_sim'].mean() >= 0.8:
    print("✓ Wysokie podobieństwo semantyczne (średnia ≥0.8)")
elif results_df['cosine_sim'].mean() >= 0.7:
    print("⚠ Średnie podobieństwo semantyczne (średnia 0.7-0.8)")
else:
    print("✗ Niskie podobieństwo semantyczne (średnia <0.7)")

In [ ]:
# ===================================================
# PRZYKŁADY (Pierwsze 5 par)
# ===================================================

print("\n" + "="*100)
print("PRZYKŁADOWE PARY TEKST-PARAFRAZA (pierwsze 5)")
print("="*100)
print()

for i, row in results_df.head(5).iterrows():
    print(f"--- Przykład {row['id']} ---")
    print(f"ORYGINAŁ: {row['original_text']}")
    print(f"PARAFRAZA: {row['paraphrase_text']}")
    print(f"Prawdop. toksyczności: {row['orig_prob']} → {row['para_prob']} (diff: {row['prob_diff']})")
    print(f"Cosine similarity: {row['cosine_sim']} {row['quality_ok']}")
    print()

In [ ]:
# ===================================================
# Mniej agresywne parafrazy
# (tylko zmiany pojednynczych słów na synonimy)
# ===================================================

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from captum.attr import IntegratedGradients

# Pobieranie niezbędnych zasobów
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger_eng')

def get_wordnet_pos(treebank_tag):
    """Mapuje tagi NLTK na format akceptowany przez WordNet."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
import random

def synonym_paraphraser(text, n_changes=2):
    """Zamienia n_changes słów na ich synonimy przy użyciu WordNet."""
    words = word_tokenize(text)
    pos_tags = nltk.pos_tag(words)

    # Indeksy słów, które mają sensowne tagi (rzeczownik, czasownik, przymiotnik, przysłówek)
    eligible_indices = [
        i for i, (word, tag) in enumerate(pos_tags)
        if get_wordnet_pos(tag) is not None and len(word) > 3
    ]

    if not eligible_indices:
        return text

    # Wybierz losowo słowa do zmiany
    to_change = random.sample(eligible_indices, min(n_changes, len(eligible_indices)))
    new_words = words.copy()

    for idx in to_change:
        word, tag = pos_tags[idx]
        wn_pos = get_wordnet_pos(tag)
        synonyms = []

        for syn in wordnet.synsets(word, pos=wn_pos):
            for lemma in syn.lemmas():
                if lemma.name().lower() != word.lower():
                    synonyms.append(lemma.name().replace('_', ' '))

        if synonyms:
            new_words[idx] = random.choice(list(set(synonyms)))

    return " ".join(new_words)

In [ ]:
from captum.attr import LayerIntegratedGradients

def get_word_attributions(text, model, tokenizer):
    """Oblicza wagi ważności dla każdego słowa, celując w warstwę embeddings."""

    # 1. Przygotowanie wejścia
    inputs = tokenizer(text, return_tensors="pt", truncation=True,
                       max_length=MAX_SEQUENCE_LENGTH).to(device)
    input_ids = inputs['input_ids']

    # 2. Definicja funkcji forward (musi zwracać tylko logity)
    def predict(ids):
        return model(ids).logits

    # 3. Wybór warstwy embeddings (w DistilBERT to model.distilbert.embeddings)
    # Jeśli używasz innego modelu, sprawdź nazwę warstwy przez print(model)
    lig = LayerIntegratedGradients(predict, model.distilbert.embeddings)

    # 4. Obliczanie atrybucji
    # target=0 oznacza, że badamy wpływ na wyjście (dla regresji/binary classification)
    attributions = lig.attribute(inputs=input_ids,
                                 target=0,
                                 n_steps=50) # n_steps to dokładność przybliżenia całki

    # 5. Agregacja: Atrybucje są dla każdego wymiaru embeddingu (np. 768),
    # musimy je zsumować, aby dostać jedną wartość na token.
    attributions = attributions.sum(dim=-1).squeeze(0)

    # Normalizacja dla lepszej porównywalności (L2)
    if torch.norm(attributions) != 0:
        attributions = attributions / torch.norm(attributions)

    # 6. Mapowanie na tokeny
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    return list(zip(tokens, attributions.cpu().detach().numpy()))

In [ ]:
import json
from scipy.stats import spearmanr

case_studies = []

print(">>> Uruchamiam test stabilności atrybucji (Synonimy)...")

for idx, row in sample_df.head(10).iterrows():  # Test na 10 przykładach
    orig_text = row['comment_text']

    # 1. Parafraza synonimiczna
    syn_text = synonym_paraphraser(orig_text, n_changes=2)

    # 2. Pobranie atrybucji (XAI)
    attr_orig = get_word_attributions(orig_text, model, tokenizer)
    attr_syn = get_word_attributions(syn_text, model, tokenizer)

    # 3. Obliczenie korelacji Spearmana (wymaga wektorów tej samej długości)
    # Dla uproszczenia: porównujemy tylko średnią ważność top-tokenów
    # lub przycinamy do krótszego tekstu
    min_len = min(len(attr_orig), len(attr_syn))
    vals_orig = [float(a[1]) for a in attr_orig[:min_len]]
    vals_syn = [float(a[1]) for a in attr_syn[:min_len]]

    correlation, _ = spearmanr(vals_orig, vals_syn)

    case_studies.append({
        "case_id": idx,
        "original": {
            "text": orig_text,
            "explanation": [{"t": t, "w": float(w)} for t, w in attr_orig]
        },
        "paraphrase_synonym": {
            "text": syn_text,
            "explanation": [{"t": t, "w": float(w)} for t, w in attr_syn]
        },
        "metrics": {
            "spearman_correlation": round(correlation, 4),
            "similarity": calculate_cosine_similarity(*get_embedding_and_prob(orig_text)[0:1],
                                                     *get_embedding_and_prob(syn_text)[0:1])
        }
    })

# Zapis do JSON (idealne do wykresów w pracy mgr)
with open('xai_case_studies.json', 'w', encoding='utf-8') as f:
    json.dump(case_studies, f, ensure_ascii=False, indent=4)

print(f"✓ Zapisano {len(case_studies)} Case Studies do pliku JSON.")

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# 1. Tabela podsumowująca metryki
summary_data = []
for study in case_studies:
    summary_data.append({
        "ID": study['case_id'],
        "Korelacja XAI (Spearman)": study['metrics']['spearman_correlation'],
        "Podobieństwo Semantyczne": round(study['metrics']['similarity'], 4),
        "Oryginał": study['original']['text'][:50] + "...",
        "Parafraza (Synonim)": study['paraphrase_synonym']['text'][:50] + "..."
    })

print("\n" + "="*50)
print("PODSUMOWANIE METRYK STABILNOŚCI")
print("="*50)
df_summary = pd.DataFrame(summary_data)
display(df_summary)

# 2. Funkcja do kolorowania tekstu na podstawie wag XAI
def colorize_text(explanation):
    """
    Renderuje tekst z kolorowaniem wag.
    Obsługuje format krotki (token, waga) oraz słownika {'t': token, 'w': waga}.
    """
    if not explanation:
        return ""

    # Konwersja do wspólnego formatu (token, waga)
    normalized_data = []
    for item in explanation:
        if isinstance(item, dict):
            normalized_data.append((item['t'], item['w']))
        else:
            normalized_data.append((item[0], item[1]))

    html_res = ""
    # Znajdujemy max wartość do skalowania intensywności kolorów
    abs_weights = [abs(w) for t, w in normalized_data]
    max_w = max(abs_weights) if abs_weights else 1

    for token, weight in normalized_data:
        # Czyszczenie subtokenów WordPiece dla lepszej czytelności
        clean_token = token.replace('##', '')

        # Obliczanie przezroczystości (intensywność koloru)
        alpha = abs(weight) / max_w

        # Czerwony: wzmacnia toksyczność | Niebieski: osłabia/neutralizuje
        color = f"rgba(255, 0, 0, {alpha:.2f})" if weight > 0 else f"rgba(0, 0, 255, {alpha:.2f})"

        html_res += f'''<span style="background-color: {color};
                                    padding: 2px 4px;
                                    margin: 1px;
                                    border-radius: 3px;
                                    display: inline-block;
                                    border: 0.5px solid rgba(0,0,0,0.1);"
                             title="Waga: {weight:.4f}">{clean_token}</span> '''
    return html_res

# 3. Wyświetlenie Case Studies side-by-side
print("\n" + "="*50)
print("WIZUALIZACJA ATKRYBUCJI (MAPY CIEPŁA)")
print("="*50)

html_output = "<div style='font-family: sans-serif;'>"
for study in case_studies[:5]: # Wyświetlamy pierwsze 5 dla przejrzystości
    html_output += f"""
    <div style="border: 1px solid #ddd; margin-bottom: 20px; padding: 10px; border-radius: 8px;">
        <h4 style="margin-top: 0;">Case ID: {study['case_id']} | Korelacja: {study['metrics']['spearman_correlation']}</h4>
        <div style="display: flex; gap: 20px;">
            <div style="flex: 1;">
                <strong>Oryginał:</strong><br>
                {colorize_text(study['original']['explanation'])}
            </div>
            <div style="flex: 1;">
                <strong>Parafraza (Synonim):</strong><br>
                {colorize_text(study['paraphrase_synonym']['explanation'])}
            </div>
        </div>
    </div>
    """
html_output += "</div>"

display(HTML(html_output))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_stability_analysis(case_studies):
    # Przygotowanie danych
    correlations = [s['metrics']['spearman_correlation'] for s in case_studies]
    similarities = [s['metrics']['similarity'] for s in case_studies]
    ids = [s['case_id'] for s in case_studies]

    plt.figure(figsize=(10, 6))
    sns.set_style("whitegrid")

    # Wykres punktowy
    scatter = plt.scatter(similarities, correlations, c=correlations, cmap='coolwarm', s=100, edgecolors='black')

    # Dodanie etykiet dla skrajnych punktów
    for i, txt in enumerate(ids):
        if correlations[i] < 0.3 or similarities[i] < 0.8: # Etykiety dla najciekawszych przypadków
            plt.annotate(txt, (similarities[i], correlations[i]), xytext=(5,5), textcoords='offset points', fontsize=9)

    plt.title('Stabilność Wyjaśnień XAI vs. Podobieństwo Semantyczne Parafraz', fontsize=14)
    plt.xlabel('Podobieństwo Semantyczne (Cosine Similarity)', fontsize=12)
    plt.ylabel('Korelacja Map Atrybucji (Spearman)', fontsize=12)
    plt.colorbar(scatter, label='Korelacja')

    # Linie pomocnicze dla interpretacji
    plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.3, label='Próg stabilności')
    plt.legend()

    plt.tight_layout()
    plt.savefig('xai_stability_plot.png', dpi=300)
    plt.show()

plot_stability_analysis(case_studies)

In [ ]:
def get_word_attributions_smoothed(text, model, tokenizer, n_samples=30, stdevs=0.05):
    """
    Oblicza wygładzone atrybucje (SmoothGrad) z poprawką na typy danych maski.
    """
    # 1. Przygotowanie wejścia
    inputs = tokenizer(text, return_tensors="pt", truncation=True,
                       max_length=MAX_SEQUENCE_LENGTH).to(device)
    input_ids = inputs['input_ids']

    # Uzyskanie bazowych embeddingów
    with torch.no_grad():
        input_embeddings = model.distilbert.embeddings(input_ids)

    # 2. Definicja funkcji forward operującej na embeddingach
    def forward_with_embeddings(embeddings):
        # KLUCZOWA POPRAWKA: Rzutujemy maskę na typ taki sam jak embeddingi (Float)
        # To eliminuje błąd RuntimeError: Expected attn_mask dtype...
        mask = inputs['attention_mask'].to(embeddings.dtype)

        # Wykorzystujemy oficjalny parametr inputs_embeds modelu
        outputs = model(inputs_embeds=embeddings, attention_mask=mask)
        return outputs.logits

    # 3. Konfiguracja XAI
    from captum.attr import IntegratedGradients, NoiseTunnel
    ig = IntegratedGradients(forward_with_embeddings)
    nt = NoiseTunnel(ig)

    # 4. Obliczanie atrybucji
    # n_steps ustawiamy na 5, aby przyspieszyć obliczenia przy wielu próbach SmoothGrad
    attributions = nt.attribute(input_embeddings,
                                 nt_type='smoothgrad',
                                 nt_samples=n_samples,
                                 stdevs=stdevs,
                                 target=0,
                                 n_steps=5)

    # 5. Agregacja i normalizacja
    attributions = attributions.sum(dim=-1).squeeze(0)

    if torch.norm(attributions) != 0:
        attributions = attributions / torch.norm(attributions)

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    return list(zip(tokens, attributions.cpu().detach().numpy()))

In [ ]:
results_comparison = []

print(">>> Testowanie poprawy stabilności przez SmoothGrad...")

# Testujemy na tych samych przykładach co wcześniej
for idx, row in sample_df.head(10).iterrows():
    orig_text = row['comment_text']
    syn_text = synonym_paraphraser(orig_text, n_changes=2)

    # 1. Standardowe IG (to co miałeś)
    attr_orig_std = get_word_attributions(orig_text, model, tokenizer)
    attr_syn_std = get_word_attributions(syn_text, model, tokenizer)

    # 2. SmoothGrad IG (nowość)
    attr_orig_smooth = get_word_attributions_smoothed(orig_text, model, tokenizer, n_samples=15)
    attr_syn_smooth = get_word_attributions_smoothed(syn_text, model, tokenizer, n_samples=15)

    # Funkcja pomocnicza do korelacji
    def get_corr(a1, a2):
        min_l = min(len(a1), len(a2))
        return spearmanr([float(x[1]) for x in a1[:min_l]],
                         [float(x[1]) for x in a2[:min_l]])[0]

    corr_std = get_corr(attr_orig_std, attr_syn_std)
    corr_smooth = get_corr(attr_orig_smooth, attr_syn_smooth)

    results_comparison.append({
        "ID": idx,
        "Korelacja Standardowa": round(corr_std, 4),
        "Korelacja SmoothGrad": round(corr_smooth, 4),
        "Poprawa": round(corr_smooth - corr_std, 4)
    })

# Wyświetlenie porównania
df_comp = pd.DataFrame(results_comparison)
display(df_comp)

print(f"\nŚrednia poprawa korelacji: {df_comp['Poprawa'].mean():.4f}")

In [ ]:
# Wizualizacja przypadku, który najbardziej zyskał (9074)
best_improvement_id = 9074

# Pobieramy wyjaśnienia obiema metodami
attr_orig_std = get_word_attributions(sample_df.loc[best_improvement_id, 'comment_text'], model, tokenizer)
attr_orig_smooth = get_word_attributions_smoothed(sample_df.loc[best_improvement_id, 'comment_text'], model, tokenizer, n_samples=30, stdevs=0.05)

html_compare = f"""
<h3>Analiza poprawy stabilności dla Case ID: {best_improvement_id}</h3>
<div style="display: flex; gap: 20px;">
    <div style="flex: 1;">
        <strong>Standard IG (Mała stabilność):</strong><br>
        {colorize_text(attr_orig_std)}
    </div>
    <div style="flex: 1;">
        <strong>SmoothGrad (Większa stabilność):</strong><br>
        {colorize_text(attr_orig_smooth)}
    </div>
</div>
"""
display(HTML(html_compare))

In [ ]:
import csv

final_results = []

print(">>> Generowanie ostatecznego zestawienia danych...")

# Łączymy wyniki z różnych testów (Mistral, Synonimy, SmoothGrad)
for study in case_studies:
    case_id = study['case_id']

    # Próbujemy znaleźć odpowiadający wynik z testu SmoothGrad
    smooth_data = next((item for item in results_comparison if item['ID'] == case_id), None)

    final_results.append({
        "ID": case_id,
        "Original_Text": study['original']['text'],
        "Paraphrase_Text": study['paraphrase_synonym']['text'],
        "Semantic_Similarity": study['metrics']['similarity'],
        "XAI_Corr_Standard": study['metrics']['spearman_correlation'],
        "XAI_Corr_SmoothGrad": smooth_data['Korelacja SmoothGrad'] if smooth_data else "N/A",
        "Improvement": smooth_data['Poprawa'] if smooth_data else 0
    })

# Zapis do CSV
output_file = 'final_xai_analysis.csv'
keys = final_results[0].keys()

with open(output_file, 'w', newline='', encoding='utf-8') as f:
    dict_writer = csv.DictWriter(f, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(final_results)

print(f"✓ Sukces! Plik '{output_file}' jest gotowy do pobrania.")